In [1]:
# Import warnings
import warnings

# Filter Warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from imblearn.ensemble import EasyEnsembleClassifier, BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')

# Read loan stats data
df = pd.read_csv(file_path) [:-2]

# Set columns to keep
df = df.loc[:, columns].copy()

# Drop the columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Save all rows except 'Issued' loans
issued_mask = df['loan_status'] != 'Issued'

# Remove the `Issued` loan status
df = df.loc[issued_mask]

# Save Low Risk values
x = {'Current': 'low_risk'}

# replace values
df = df.replace(x)

# Save High Risk Values
x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    

# Replace values
df = df.replace(x)

# Reset Index
df.reset_index(inplace=True, drop=True)

# Show DataFrame
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# Create Target
y = df[["loan_status"]]

# Drop Target
X = df.drop(columns="loan_status")

# Encoding the Bank, State and City columns
X = pd.get_dummies(X)

# Describe
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68815.000000,68815.000000,68815.000000,6.881500e+04,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,...,68815.000000,68815.0,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,68815.000000,68815.0,68815.0
mean,16677.265494,0.127719,480.643936,8.820767e+04,21.778539,0.217743,0.497697,12.587227,0.126019,17603.616741,...,0.177243,1.0,0.123868,0.876132,0.383172,0.616828,0.860336,0.139664,1.0,1.0
std,10277.113051,0.048130,288.052205,1.155699e+05,20.199409,0.718345,0.758128,6.022871,0.336785,21835.601935,...,0.381877,0.0,0.329434,0.329434,0.486163,0.486163,0.346641,0.346641,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.895000,0.000000,0.000000,8.000000,0.000000,6292.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,647.950000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21734.500000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [6]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68468
high_risk      347
Name: loan_status, dtype: int64

In [7]:
# Split the X and y into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51611, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Create BalancedRandomForestClassifier instance
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit Model
brfc.fit(X_train_scaled,y_train)

BalancedRandomForestClassifier(random_state=1)

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
# Predict
y_predict = brfc.predict(X_test_scaled)

# Save Accuracy Score for BalancedRandomForestClassifier
brfc_bac = balanced_accuracy_score(y_test, y_predict)

# Show Score
brfc_bac

0.7940811682141636

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, y_predict)

array([[   61,    26],
       [ 1934, 15183]])

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.89      0.06      0.79      0.61        87
   low_risk       1.00      0.89      0.70      0.94      0.79      0.63     17117

avg / total       0.99      0.89      0.70      0.93      0.79      0.63     17204



In [18]:
# Save Naive Random Oversampling Recall
brfc_recall = classification_report_imbalanced(y_test, y_predict).split()[27]

# Save Naive Random Oversampling Geometric Mean Score
brfc_geo = classification_report_imbalanced(y_test, y_predict).split()[30]

In [19]:
# List the features sorted in descending order by feature importance
# Feature importances
importances = brfc.feature_importances_

feature_list = sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

feature_list

[(0.08363922556976805, 'total_rec_prncp'),
 (0.059562502736071665, 'total_pymnt_inv'),
 (0.055699120164476514, 'last_pymnt_amnt'),
 (0.0552141131828394, 'total_pymnt'),
 (0.055064927060282445, 'total_rec_int'),
 (0.03057843396499587, 'int_rate'),
 (0.021052111043326337, 'dti'),
 (0.018273791333692713, 'issue_d_Jan-2019'),
 (0.01777999292391785, 'mo_sin_old_il_acct'),
 (0.017742662502271186, 'out_prncp'),
 (0.01638770738504851, 'il_util'),
 (0.015795145465440627, 'tot_cur_bal'),
 (0.015305940338186556, 'all_util'),
 (0.015226054737394303, 'max_bal_bc'),
 (0.014709474550757209, 'installment'),
 (0.014587911128108363, 'tot_hi_cred_lim'),
 (0.014211102438843359, 'revol_bal'),
 (0.014095666199301459, 'total_bal_ex_mort'),
 (0.013993261481266601, 'annual_inc'),
 (0.013929133771766111, 'bc_open_to_buy'),
 (0.013918364438008268, 'avg_cur_bal'),
 (0.013886715308382544, 'bc_util'),
 (0.013789849872699602, 'mths_since_recent_inq'),
 (0.01370731281805038, 'mo_sin_old_rev_tl_op'),
 (0.0129821347444

### Easy Ensemble Classifier

In [20]:
# Train the EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fit
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [22]:
# Calculate the balanced accuracy score
# Predict
y_predict = eec.predict(X_test_scaled)

# Save EasyEnsembleClassifier Balanced Accuracy Score
eec_bac = balanced_accuracy_score(y_test, y_predict)

# Score
eec_bac

0.9352257183320474

In [96]:
# Display the confusion matrix
confusion_matrix(y_test, y_predict)

array([[   81,     6],
       [ 1037, 16080]])

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.93      0.94      0.13      0.94      0.87        87
   low_risk       1.00      0.94      0.93      0.97      0.94      0.88     17117

avg / total       0.99      0.94      0.93      0.96      0.94      0.88     17204



In [25]:
# Save EEC Recall
eec_recall = classification_report_imbalanced(y_test, y_predict).split()[27]

# Save EEC Geometric Mean Score
eec_geometric_mean = classification_report_imbalanced(y_test, y_predict).split()[30]

# Dataframe for scores
scores = pd.DataFrame({"Balanced Accuracy Score": [brfc_bac, eec_bac], 
                       "Recall Score": [brfc_recall, eec_recall], 
                       "Geometric Mean Score": [brfc_geo, eec_geometric_mean]},
                                    index=["BRFC", "EEC"])

In [31]:
# Finding the best score
def findBest(COLUMN):
    # sort values
    best = scores.sort_values(COLUMN, ascending=False)
    
    # save index for top value
    best = best.index[0]
    
    #return saved index
    return best

# Save best BAC Score using findBest
best_bac = findBest("Balanced Accuracy Score")

# Save best Recall Score
best_recall = findBest("Recall Score")

# Save best Geometric Mean Score
best_geo = findBest("Geometric Mean Score")

# Empty list for top feature importances
top_fi = []

# Iterate top 3
for x in range(3):
        # append feature to list
        top_fi.append(feature_list[x][1])
    
# print analysis
print(f"""{best_bac} has the best balanced accuracy score.
{best_recall} has the best recall score.
{best_geo} has the best geometric mean score.
The top 3 features are {top_fi[0]}, {top_fi[1]}, and {top_fi[2]}.""")

EEC has the best balanced accuracy score.
EEC has the best recall score.
EEC has the best geometric mean score.
The top 3 features are total_rec_prncp, total_pymnt_inv, and last_pymnt_amnt.
